In [45]:
import numpy as np
import math
from itertools import product
#computes the entropy of a probability distribution given a list of probabilities "probabilities"
def H(probabilities):
    sum = 0.0
    for i in probabilities:
        if (i <= 0 and i>= -0.01):
            sum += 0
        else:
            sum -= i * math.log2(i)
    return sum

In [4]:
H([0.1, 0.9])

0.4689955935892812

In [335]:
#computes the objective function given numpy arrays p, a, b
def loss(p, a, b):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += p[i] * (H([a[i], 1 - a[i]]) + H([b[i], 1 - b[i]]))
    return sum

In [6]:
def h(x):
    return H([x, 1-x])

In [7]:
h(0.1)

0.4689955935892812

In [8]:
def L(x):
    return h(x) + 1 - x

In [336]:
# This computes the left hand side of the inequality in the conditions, given numpy arrays p, a, b, c (outdated)
def oldLHS(p, a, b, c):
    n = len(np.atleast_1d(p))
    sum1 = 0
    sum2 = 0
    sum3 = 0
    for i in range(n):
        sum1 += p[i] * (a[i] * b[i] - c[i])
        sum2 += p[i] * (a[i] * (1 - b[i]) + (1 - a[i]) * b[i] + 2 * c[i])
        sum3 += p[i] * ((1 - a[i]) * (1 - b[i]) - c[i])
    return H([sum1, sum2, sum3])

In [337]:
def LHS(p, a, b, c):
    n = len(np.atleast_1d(p))
    sum = 0
    for i in range(n):
        sum += (p[i] * (a[i] * (1 - b[i]) + b[i] * (1 - a[i]) + 2 * c[i]))
    if (sum < 0 or sum > 1):
        return False
    return L(sum)

In [353]:
# This computes the right hand side of the inequality in the conditions, given numpy arrays p, a, b, c

def RHS(p, a, b, c):
    entropy_sum = 0
    n = len(np.atleast_1d(p))
#     print(a)
#     print(b)
#     print(c)
    for i in range(n):
        if ((a[i] * b[i] - c[i]) < 0 or (a[i] * (1 - b[i]) + c[i]) < 0 or (b[i] * (1 - a[i]) + c[i]) < 0 or ((1 - a[i]) * (1 - b[i]) + c[i]) < 0):
            return False

    for i in range(n):
        entropy = H([a[i] * b[i] - c[i], a[i] * (1 - b[i]) + c[i], (1 - a[i]) * b[i] + c[i], (1 - a[i]) * (1 - b[i]) - c[i]])
        entropy_sum += p[i] * entropy
    return entropy_sum

In [339]:
# This computes quantity to minimize for each value of c, given numpy arrays p, a, b, c

def LHS_minus_RHS(p, a, b, c):
    if (LHS(p,a,b,c) == False or RHS(p,a,b,c) == False):
        return False
    return LHS(p, a, b, c) - RHS(p, a, b, c)

In [340]:
#Uses gradient descent to find the worst values of c possible

def worst_c(p, a, b, alpha, iterations):
    n = len(np.atleast_1d(p))
    c = np.zeros(n, dtype = float)
    for i in range(iterations):
        for j in range(n):
            cplus = list(c)
            cplus[j] += 0.001
            cminus = list(c)
            cminus[j] -=  0.001
            dxdc = (LHS_minus_RHS(p, a, b, cplus) - LHS_minus_RHS(p, a, b, cminus))/.002
            c[j] -= dxdc * alpha
        for j in range(n):
            if (a[j] * b[j] - c[j] < 0 or a[j] * (1 - b[j]) + c[j] < 0 or b[j] * (1 - a[j]) + c[j] < 0 or (1 - a[j]) * (1 - b[j]) + c[j] < 0):
                c += dxdc * alpha
                return c
            
    return c

In [293]:
%%time
p = np.array((1.0/3, 1.0/3, 1.0/3))
atest = np.array((0.2, 0.5, 0.2))
btest = np.array((0.3, 0.3, 0.4))
c = worst_c(p, atest, btest, alpha = 0.01, iterations = 10)
# c = np.array((1,1,1))
print(LHS_minus_RHS(p, atest, btest, c))
print(c)

-0.18787224446955308
[-0.00598564  0.03217907  0.0057064 ]
CPU times: user 11.5 ms, sys: 795 µs, total: 12.2 ms
Wall time: 11.5 ms


In [267]:
%%time
p = np.array((0.05, 0.05, 0.9))
atest = np.array((0.2, 0.5, 0.2))
btest = np.array((0.3, 0.3, 0.4))
c = worst_c(p, atest, btest, alpha = 0.001, iterations = 1000)
# c = np.array((1,1,1))
print(LHS_minus_RHS(p, atest, btest, c))
print(c)

-0.1504782038516299
[-0.00624847  0.03743791  0.00728268]
CPU times: user 627 ms, sys: 4.07 ms, total: 631 ms
Wall time: 637 ms


In [358]:
# Ignore for now
#There are issues when a * (1-b) is very close to 0
a = np.array((0.2, 0.2, 0.2))
b = np.array((0.2, 0.2, 0.2))
p = np.array((0.2, 0.4, 0.4))
c = worst_c(p, a, b, alpha = 0.001, iterations = 10000 )
print(c)
sumLHS1 = 0
sumLHS2 = 0
sumLHS3 = 0
for i in range(3):
    sumLHS1 += (p[i] * (a[i] * b[i] - c[i]))
    sumLHS2 += (p[i] * ((1 - a[i]) * (1 - b[i]) - c[i]))
    sumLHS3 += (p[i] * (a[i] * (1 - b[i]) + b[i] * (1 - a[i]) + 2 * c[i]))
LHSval = sumLHS1 * sumLHS2 / (sumLHS3 * sumLHS3)
# print(sumLHS1)
# print(sumLHS2)
# print(sumLHS3)
print(LHSval)
x = 2

RHSval = (a[x] * b[x] - c[x]) * ((1 - a[x]) * (1 - b[x]) - c[x])/(((1 - a[x]) * b[x] + c[x]) * ((1 - b[x]) * a[x] + c[x]))
print(RHSval)

[-0.0064139  -0.00641393 -0.00641393]
0.31797672384634895
1.271907241922548


In [306]:
# %%time
# #DO NOT RUN THIS YET, EVERYTHING ELSE NEEDS TO BE FIXED FIRST

# x = np.linspace(0.0, 0.5, num = 6)
# ptest = np.array((1.0/3, 1.0/3, 1.0/3))
# maxloss = 0.0
# pbest = 0
# abest = 0
# bbest = 0
# cbest = 0

# acheck = 0
# bcheck = 0

# for (a1, a2, a3, b1, b2, b3) in product(x, x, x, x, x, x):
#     a = [a1, a2, a3]
#     b = [b1, b2, b3]
#     c = worst_c(ptest, a, b, alpha = 0.01, iterations = 10)
#     for (p_1, p_2) in product(x, x):
#         p_3 = 1 - p_1 - p_2
#         if (p_3 < 0):
#             continue
#         p = [p_1, p_2, p_3]
#         if LHS_minus_RHS(p, a, b, c) >= 0:
#             if (maxloss < loss(p, a, b)):
#                 c = worst_c(p, a, b, alpha = 0.001, iterations = 1000)
#                 if LHS_minus_RHS(p, a, b, c) >= 0:
#                     pbest = p
#                     abest = a
#                     bbest = b
#                     cbest = c
#                     maxloss = loss(p, a, b)
# print(maxloss)

1.5723829967913763
CPU times: user 11min 4s, sys: 8.85 s, total: 11min 13s
Wall time: 12min 22s


In [307]:
print(pbest)
print(abest)
print(bbest)
print(cbest)

[0.2, 0.2, 0.6000000000000001]
[0.0, 0.30000000000000004, 0.30000000000000004]
[0.2, 0.4, 0.30000000000000004]
[ 0.         -0.00889281 -0.02466034]


In [311]:

c = worst_c(pbest, abest, cbest, alpha = 0.001, iterations = 1000)
print(c)
print(LHS_minus_RHS(pbest, abest, bbest, cbest))

[0. 0. 0.]
4.82001024346701e-05


In [357]:
a = np.array((0.23684, 0.23684, 0.23684))
b = np.array((0.23684, 0.23684, 0.23684))
c = np.array((0.0153821919, 0.0153821919, 0.0153821919))
p = np.array((0.2, 0.8))
print(worst_c(p, a, b, alpha = 0.001, iterations = 1000))

[0.01537418 0.01537666]


In [356]:
LHS_minus_RHS(p, a, b, worst_c(p, a, b, alpha = 0.001, iterations = 1000))

-8.511719146397922e-06

In [355]:
LHS_minus_RHS(p, a, b, c)

-8.512262101856294e-06

In [350]:
LHS(p, a, b, c)

1.5739832616659277

In [354]:
RHS(p, a, b, c)

1.5739917739280296